In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

In [15]:
!pip install -U scikit-learn

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 397, in resolve
    self._add_to_criteria(self.state.criteria, r, parent=None)
  File "/usr/local/lib/pytho

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
%cd /content/drive/MyDrive/pipe/lab4sem

/content/drive/MyDrive/pipe/lab4sem


In [18]:
df = pd.read_csv('Laptop_price.csv')

In [19]:
data = pd.read_csv('Laptop_price.csv')

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand             1000 non-null   object 
 1   Processor_Speed   1000 non-null   float64
 2   RAM_Size          1000 non-null   int64  
 3   Storage_Capacity  1000 non-null   int64  
 4   Screen_Size       1000 non-null   float64
 5   Weight            1000 non-null   float64
 6   Price             1000 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 54.8+ KB


In [141]:
data.set_index('Price', inplace=True)

In [21]:

y = data[['Price']]

In [22]:
X = data.drop('Price', axis=1)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)

In [24]:
num_columns = list(X_train.select_dtypes(exclude=['object']).columns)
num_columns

['Processor_Speed', 'RAM_Size', 'Storage_Capacity', 'Screen_Size', 'Weight']

In [25]:
cat_columns = list(X_train.select_dtypes(include=['object']).columns)
cat_columns

['Brand']

In [51]:
numerical = Pipeline(steps=[
    ("simtle_imputer", SimpleImputer(strategy='constant')),
    ("power_transform", PowerTransformer()),
    ('Scaler',StandardScaler())
])

In [52]:
categorical = Pipeline(steps=[
    ("simple_limiter", SimpleImputer(strategy='constant')),
    ("OneHotEncoder", OneHotEncoder(handle_unknown='ignore',
                                    sparse_output=False))
])

In [53]:
ct = ColumnTransformer([
    ("numerical", numerical, num_columns ),
     ("categorical", categorical, cat_columns )
])

In [54]:
pipe = Pipeline(steps=[
    ("ct", ct),
    ("XGBRegressor", XGBRegressor())
])

In [59]:
param_grid={
    'XGBRegressor__n_estimators' : [50],
    'XGBRegressor__learning_rate' : [1],
    'XGBRegressor__max_depth' : [3],
    'XGBRegressor__gamma' : [6],
    'XGBRegressor__max_leaves' : [0],
    'ct__numerical' : [StandardScaler(), RobustScaler()]
}

In [60]:
clf = GridSearchCV(pipe, param_grid, verbose=3)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END XGBRegressor__gamma=6, XGBRegressor__learning_rate=1, XGBRegressor__max_depth=3, XGBRegressor__max_leaves=0, XGBRegressor__n_estimators=50, ct__numerical=StandardScaler();, score=0.999 total time=   0.0s
[CV 2/5] END XGBRegressor__gamma=6, XGBRegressor__learning_rate=1, XGBRegressor__max_depth=3, XGBRegressor__max_leaves=0, XGBRegressor__n_estimators=50, ct__numerical=StandardScaler();, score=0.999 total time=   0.0s
[CV 3/5] END XGBRegressor__gamma=6, XGBRegressor__learning_rate=1, XGBRegressor__max_depth=3, XGBRegressor__max_leaves=0, XGBRegressor__n_estimators=50, ct__numerical=StandardScaler();, score=0.999 total time=   0.0s
[CV 4/5] END XGBRegressor__gamma=6, XGBRegressor__learning_rate=1, XGBRegressor__max_depth=3, XGBRegressor__max_leaves=0, XGBRegressor__n_estimators=50, ct__numerical=StandardScaler();, score=0.999 total time=   0.0s
[CV 5/5] END XGBRegressor__gamma=6, XGBRegressor__learning_rate=1, XGBRe

GridSearchCV(estimator=Pipeline(steps=[('ct',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         Pipeline(steps=[('simtle_imputer',
                                                                                          SimpleImputer(strategy='constant')),
                                                                                         ('power_transform',
                                                                                          PowerTransformer()),
                                                                                         ('Scaler',
                                                                                          StandardScaler())]),
                                                                         ['Processor_Speed',
                                                                          'RAM_Size',
                                                                          'Storage_Capacity',
                                                                          'Screen_Size',
                                                                          'Weight']),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('simple_limiter...
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             param_grid={'XGBRegressor__gamma': [6],
                         'XGBRegressor__learning_rate': [1],
                         'XGBRegressor__max_depth': [3],
                         'XGBRegressor__max_leaves': [0],
                         'XGBRegressor__n_estimators': [50],
                         'ct__numerical': [StandardScaler(), RobustScaler()]},
             verbose=3)

In [61]:
model=clf.best_estimator_

In [62]:
clf.best_params_

{'XGBRegressor__gamma': 6,
 'XGBRegressor__learning_rate': 1,
 'XGBRegressor__max_depth': 3,
 'XGBRegressor__max_leaves': 0,
 'XGBRegressor__n_estimators': 50,
 'ct__numerical': StandardScaler()}